# 04bis - Estimation bayésienne du paramètre xi

Dans ce notebook, nous allons essayer de mettre en place une estimation bayésienne pour le parmaètre xi à l'ensemble des stations du québec, après l'estimation normale.

In [1]:
using CSV, DataFrames, StatsBase, Random

using Distributions, Extremes, Optim, LinearAlgebra, Distributions

using Mamba, ForwardDiff, ProgressMeter #pour les MCMC

using IDF

import Plots

In [2]:
PROVINCES = ["NB", "NL", "NS", "ON", "PE", "QC"]#provinces considerees

DURATION = "24 h"

"24 h"

In [3]:
station_list = load_data("station_list")

filter!(row -> row.Province ∈ PROVINCES , station_list)#on ne selectionne que les stations qui nous interessent

first(station_list, 10)

,Name,Province,ID,Lat,Lon,Elevation
,String,String,String,Float64,Float64,Int64
1,BEECHWOOD,NB,8100512,46.53,-67.67,91
2,BELLEDUNE,NB,8100514,47.9,-65.83,7
3,BOUCTOUCHE CDA CS,NB,8100593,46.43,-64.77,35
4,CHARLO AUTO,NB,8100885,47.98,-66.33,42
5,MIRAMICHI RCS,NB,8100989,47.02,-65.47,33
6,EDMUNDSTON,NB,8101303,47.42,-68.32,154
7,FREDERICTON A,NB,8101500,45.87,-66.53,20
8,FREDERICTON CDA CS,NB,8101605,45.92,-66.62,35
9,MONCTON INTL A,NB,8103201,46.12,-64.68,70


Comme l'estimation classique n'a pas fonctionné à cause du nombre important de paramètres, nous allons donc procéder à une estimation bayésienne, avec comme paramètres initiaux ceux obtenus par une estimation de maximum de vraisemblance pour une loi de Gumbel (c.a.d. avec $\xi = 0$).

Commençons donc par construire ce tableau de valeurs initiales.

In [4]:
# same function as in IDF package, built it before integrating to IDF.jl
function solve_gumbel(y::Vector{Float64})
    μ₀ = mean(y)
    ϕ₀ = log(std(y))
    p₀ = [μ₀, ϕ₀]
    
    # la fonction objectif, comme on maximise la log_vraisemblance
    # il faut prendre l'oppose car optimize.jl minimise.
    function fobj(p::Vector{Float64})
        return -logL(y, p[1], exp(p[2]), 0.0)
    end
    
    
    p = p₀
    
    try
        res = optimize(fobj, p₀)
        
         
        if Optim.converged(res)
            p = Optim.minimizer(res)
        else
            @warn "The maximum likelihood algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values."
            p = p₀
        end
        
    catch
        println("Error of execution")
    end
   
    return p
end
    
μ₀ = []
ϕ₀ = []

for i in 1:(nrow(station_list))
    y = pcp(station_list[i, :ID], DURATION)
    
    p = solve_gumbel(y)
    
    append!(μ₀, p[1])
    append!(ϕ₀, p[2])
end

station_list[:μ₀] = Float64.(μ₀)
station_list[:ϕ₀] = Float64.(ϕ₀)

first(station_list, 10)

,Name,Province,ID,Lat,Lon,Elevation,μ₀,ϕ₀
,String,String,String,Float64,Float64,Int64,Float64,Float64
1,BEECHWOOD,NB,8100512,46.53,-67.67,91,55.1486,2.70978
2,BELLEDUNE,NB,8100514,47.9,-65.83,7,40.2054,2.39197
3,BOUCTOUCHE CDA CS,NB,8100593,46.43,-64.77,35,53.4377,2.57062
4,CHARLO AUTO,NB,8100885,47.98,-66.33,42,48.4714,2.66403
5,MIRAMICHI RCS,NB,8100989,47.02,-65.47,33,49.9964,2.78772
6,EDMUNDSTON,NB,8101303,47.42,-68.32,154,48.2441,2.79296
7,FREDERICTON A,NB,8101500,45.87,-66.53,20,50.8524,2.61132
8,FREDERICTON CDA CS,NB,8101605,45.92,-66.62,35,55.2421,2.73373
9,MONCTON INTL A,NB,8103201,46.12,-64.68,70,51.9352,2.87873


In [5]:
θ₀ = Float64.([station_list[:μ₀]; station_list[:ϕ₀]; 0.0])

673-element Vector{Float64}:
 55.14855306528422
 40.20542553881306
 53.43774890879652
 48.47140573289281
 49.996392488014884
 48.24412232109405
 50.852392666251404
 55.242107889525414
 51.93524893967279
 49.83618162207249
 51.209661078586855
 62.756030020231165
 67.97254136984631
  ⋮
  2.387908247372233
  2.1043510431058565
  2.287271290366869
  2.2640535479064514
  2.273393761089009
  2.0390156967528696
  2.010442168292835
  2.377784759339927
  2.131497963704006
  1.6611828658787275
  2.3630729736686393
  0.0

Maintenant que nous avons les valeurs initiales, lançons l'estimation bayésienne.

In [6]:
function xi_bayes(stations_df::DataFrame, niter::Int=5000, warmup::Int=2000)
    
    # ---- valeurs initiales
    N = nrow(stations_df)
    initialvalues = Float64.([stations_df[:μ₀]; stations_df[:ϕ₀]; 0.0])
    
    
    # ---- definition des fonctions qui serviront à la construction de la chaîne
    
    # la fonction de log vraisemblance est celle vue au notebook 4
    function logf(θ::Vector{Float64})
        μ = θ[1:N]
        ϕ = θ[N+1:(2*N)]
        ξ = θ[2*N+1]

        s=0
        for i in 1:N
            id = stations_df[i, :ID]
            s -= logL(pcp(id, DURATION), μ[i], exp(ϕ[i]), ξ) # on met l'opposé de la log-vraisemblance
        end

        return s
    end
    
    Δlogf(θ::Vector{Float64}) = ForwardDiff.gradient(logf, θ)
    
    function logfgrad(θ::Vector{Float64})
        ll = logf(θ)
        g = Δlogf(θ)
        return ll, g
    end
    
    # ---- lancement de la simulation par MCMC
    
    sim = Chains(niter, 2*N+1, start = (warmup + 1))
    θ = NUTSVariate(initialvalues, logfgrad)
    @showprogress for i in 1:niter
        sample!(θ, adapt = (i <= warmup))
        if i > warmup
            sim[i, :, 1] = θ
        end
    end
    
    # ---- renvoi de la chaîne
    
    return sim
    
end

xi_bayes (generic function with 3 methods)

In [7]:
xi_bayes(station_list)

LoadError: MethodError: no method matching (::var"#logf#4"{DataFrame, Int64})(::Vector{ForwardDiff.Dual{ForwardDiff.Tag{var"#logf#4"{DataFrame, Int64}, Float64}, Float64, 12}})
[0mClosest candidates are:
[0m  (::var"#logf#4")([91m::Vector{Float64}[39m) at In[6]:11

Les fonctions du package Mamba.jl semblant chancelantes pour des estimations multivariées, nous allons concevoir nous même l'algorithme. Nous allons donc pour ceci établir un algorithme d'échantillonage de Gibbs.

In [8]:
function logV(Y::Vector{Float64}, p::Vector{Float64})
    return logL(Y, p[1], exp(p[2]), p[3])
end

logV (generic function with 1 method)

In [18]:
function gibbs(stations_df::DataFrame, δ::Real=0.5, niter::Int=5000, warmup::Int=2000)
    
    # ---- valeurs initiales
    N = nrow(stations_df)

    Y = Vector{Float64}[]

    # for i=1:size(station_list,1)
    for stationID in stations_df[:,:ID]
    #     stationID = station_list[i,:ID]
        y = IDF.pcp(stationID, DURATION)
        push!(Y, y)
    end

    μ = zeros(N, niter)
    ϕ = zeros(N, niter)
    ξ = zeros(niter)


    μ[:, 1] = stations_df[:μ₀]
    ϕ[:, 1] = stations_df[:ϕ₀]
    ξ[1] = 0.0

    acc = 0 # nombre d'acceptations
    tot = 0
    

    # échantillonage
    
    @showprogress for iter = 2:niter
        
        
        for i in 1:N
            u = rand(N)

            # ----- estimation de mu
            
            
            # on va estimer avec Metropolis Hastings, en utilisant une loi normale N([μ₀, ϕ₁, ξ], δ * I^-1) 
            m = [μ[i, iter - 1], ϕ[i, iter - 1], ξ[iter - 1]]
            cov = δ * Matrix(I, 3, 3)
            candidates = rand(MvNormal(m, cov))
            ll = logV(Y[i], candidates) - logV(Y[i], m)

            if ll > log(u[i])
                μ[i, iter] = candidates[1]
                acc += 1
            else
                μ[i, iter] = μ[i, iter - 1]
            end
            tot +=1

            # ----- estimation de phi
            
            m = [μ[i, iter], ϕ[i, iter - 1], ξ[iter - 1]]
            cov = δ * Matrix(I, 3, 3)
            candidates = rand(MvNormal(m, cov))
            ll = logV(Y[i], candidates) - logV(Y[i], m)

            if ll > log(u[i])
                ϕ[i, iter] = candidates[1]
                acc += 1
            else
                ϕ[i, iter] = ϕ[i, iter - 1]
            end
            tot += 1
        end

        u = rand(N)
        m = [μ[:, iter] ; ϕ[:, iter] ; ξ[iter - 1]]
        M = length(m)
        cov = δ * Matrix(I, M, M)
        candidates = rand(MvNormal(m, cov))
        
        ll = 0
        threshold = 0

        for k in 1:N
            params_candidates = [candidates[k], candidates[k + N], candidates[M]]
            params_prev = [m[k], m[k + N], candidates[M]]
            
            ll += logV(Y[k], params_candidates) - logV(Y[k], params_prev)
            threshold += log(u[k])
        end


        if ll > threshold
            ξ[iter] = candidates[M]
            acc += 1
        else
            ξ[iter] = ξ[iter - 1]
        end
        
        tot += 1

    end

    rate = acc/tot
    println("The rate of acceptance is $rate")
    return [μ, ϕ, ξ]
    
end



gibbs (generic function with 4 methods)

In [19]:
df_PE = filter(row -> row.Province == "PE" , station_list)
par_PE = gibbs(df_PE)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


The rate of acceptance is 0.41816934815534534


3-element Vector{Array{Float64, N} where N}:
 [54.120899472986025 54.120899472986025 … 5.003261725504753 5.003261725504753; 42.41115376402803 42.41115376402803 … 6.740393870414034 6.740393870414034; 51.376176259690524 51.376176259690524 … 36.06970047679857 36.06970047679857]
 [2.8703714188233618 2.8703714188233618 … 4.37099225160508 4.37099225160508; 2.40421709810781 2.40421709810781 … 7.040750364685434 7.040750364685434; 2.6339652488626477 2.6339652488626477 … 36.098905967212694 36.098905967212694]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  -13.18221768883898, -13.18221768883898, -13.18221768883898, -13.18221768883898, -13.18221768883898, -13.18221768883898, -13.18221768883898, -13.18221768883898, -13.18221768883898, -13.18221768883898]

In [21]:
df_QC = filter(row -> row.Province == "QC", station_list)
par_QC = gibbs(df_QC)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:08


The rate of acceptance is 0.44160060082191877


3-element Vector{Array{Float64, N} where N}:
 [50.861683870711815 50.861683870711815 … 104.69741766275244 104.69741766275244; 52.732108761093755 52.732108761093755 … 45.93901007506271 44.14662788288113; … ; 19.256234854456228 19.256234854456228 … 2.632310525702106 2.632310525702106; 32.825126828737645 32.825126828737645 … 4.487110562176809 4.487110562176809]
 [1.8752833687464232 50.26065785207065 … 105.23581396909174 105.3434893453531; 2.552466693890037 2.552466693890037 … 46.991439618120054 44.49565018475631; … ; 1.6611828658787275 17.92804585600518 … 1.9424435680169199 1.9424435680169199; 2.3630729736686393 2.3630729736686393 … 4.057665735449468 4.057665735449468]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  -6.054821348463242, -6.054821348463242, -6.054821348463242, -6.054821348463242, -6.054821348463242, -6.054821348463242, -6.054821348463242, -6.054821348463242, -6.054821348463242, -6.054821348463242]

In [22]:
gibbs(station_list)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:39


The rate of acceptance is 0.5287833198140371


3-element Vector{Array{Float64, N} where N}:
 [55.14855306528422 55.14855306528422 … 67.36264558623516 67.60579517933374; 40.20542553881306 40.20542553881306 … 95.04207989238742 95.04207989238742; … ; 19.256234854456228 19.256234854456228 … 53.297788905862284 53.33520537694366; 32.825126828737645 32.825126828737645 … 45.5384295081349 45.5384295081349]
 [2.7097848836495837 2.7097848836495837 … 67.71976017051333 67.77425620942188; 2.3919719572617533 2.3919719572617533 … 95.01015107938434 95.01015107938434; … ; 1.6611828658787275 19.12550610151535 … 52.09950725924079 52.65812022014095; 2.3630729736686393 2.3630729736686393 … 46.22710870564951 46.22710870564951]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  55.94454158869993, 55.65437164317254, 56.03733644164725, 56.14728498124429, 55.26584293963565, 57.07654717844677, 56.78036004735608, 57.43045745659332, 57.43417486023264, 55.84664844988536]